In [9]:
import json
import csv
import pandas as pd
import requests

In [10]:
def readfile(filepath= "./data_json/cap[665].json"):
    with open(filepath, 'r') as json_file:
        return json.load(json_file)


In [11]:
from pydantic import BaseModel

class City(BaseModel):
    name: str
    region: str
    country: str
    latitude: float
    longitude: float
    utcOffset: float = 1.0
    zone: str = "Africa/Lagos"
    


class Astronomy(BaseModel):
    city: str = ""
    date: str = ""
    sunrise: str
    sunset: str
    moonrise: str
    moonset: str
    moon_phase: str
    moon_illumination: int
    
    
class Weather(BaseModel):
    city: str = ""
    date: str = ""  
    maxtempC: float
    maxtempF: float
    mintempC: float
    mintempF: float
    avgtempC: float
    avgtempF: float
    totalSnow_cm: float
    sunHour: float
    uvIndex: float
    

In [12]:
from typing import Optional
class ParseFile:
    def __init__(self, json_handler: Optional[dict]):
        
        if type(json_handler) != dict:
            json_file = json_handler()
        else:
            json_file = json_handler
        
        self.json_data = json_file.get('data')
        self.time_zone = self.json_data['time_zone'][0]
        self.weather: Optional[Weather] = None
        self.astronomy: Optional[Astronomy] = None

    def parse_area(self) -> City:
        """
        ------------
        return type: dict
        ------------

        """

        area = self.json_data['nearest_area'][0]

        area_info = []
        for ar in area.items():
            ar = list(ar)
            if ar[0] == 'areaName':
                ar[0] = 'name'
            if type(ar[1]) == list:
                ar[1] = ar[1][0]['value']
            area_info.append(ar)
        
        area_info = dict(area_info)
        

        self.city = area_info['name']
        area_info['zone'] = self.time_zone['zone']
        area_info['utcOffset'] = self.time_zone['utcOffset']
        city = City.parse_obj(area_info)
        return city

    
    def parse_weather(self) -> tuple:
        if not (self.weather and self.astronomy):

            weather_dict = self.json_data.get('weather')[0]  
            weather_dict.pop('date')
            weather_dict['city'] = self.city
            weather_dict['date'] = self.time_zone['localtime']
            hourly = weather_dict.pop('hourly')[0]
            hourly['city'] = self.city
            hourly['time'] = self.time_zone['localtime']
            
            #weather_dict['hourly'] = hourly

            astronomy_dict = weather_dict.pop('astronomy')[0]
            astronomy = Astronomy.parse_obj(astronomy_dict)
            astronomy.city = self.city
            astronomy.date = self.time_zone['localtime']

            

            weather = Weather.parse_obj(weather_dict)
            self.weather = weather
            self.astronomy = astronomy
            self.hourly = hourly

        return self.weather, self.astronomy, self.hourly

                
            
            
            

In [13]:

from datetime import datetime
import os


def to_csv(filename: str, data):

    dirname = "./weather" + datetime.now().strftime("%Y-%m-%d-%h")
    if not os.path.isdir(dirname):
        os.mkdir(dirname)

    filepath = dirname+ "/" + filename
    if os.path.isfile(filepath):
        os.remove(filepath)

    with open(filepath, 'a+') as fp:
        writer = csv.DictWriter(fp, data)
        writer.writeheader()
        writer.writerow( data)

In [ ]:
import boto3
from io import StringIO
import os
import pandas as pd


def create_filestreams(data):

    """Create file streams for city, wetaher, astronomy, and hourly"""

    global streams, writers, files
    streams = {}
    writers = {}
    files = ['city', 'weather', 'astronomy', 'hourly']
    for fl in files:

        streams[fl] = (StringIO(),)
        writers[fl] = csv.DictWriter(streams[fl], data)
        writers[fl].writeheader()

def load_file(data):
    for fl in files:
        writers[fl].writerow(data)

def upload(filename, file, bucket="weather-ng"):
    

def to_s3(filename, file, bucket="weather-ng"):
    s3_resource = boto3.resource('s3')
    res = s3_resource.Object(bucket, filename).put(Body=file)
    if res['ResponseMetadata']['HTTPStatusCode'] == 200:
        return True

In [14]:
from data_api import gen_state

def process(upload=False):

    state = pd.read_csv("list_of_capitals.csv").dropna()
    capitals = state['Capital'].values

    streams = {}
    writers = {}
    files = ['city', 'weather', 'astronomy', 'hourly']

    # Extract
    weather_data_gen = iter(gen_state(capitals))
    while True:
        
        try:
            jsondata = next(weather_data_gen)
            # Transform
            parser = ParseFile(json_handler=jsondata)

            city = parser.parse_area()
            weather, astronomy, hourly  = parser.parse_weather()

        # Load (to csv)
        if upload == True:
            to_s3('city.csv',city.dict())
            to_s3('weather.csv', weather.dict())
            to_s3('astronomy.csv', astronomy.dict())
            to_s3('metadata.csv', hourly)

        else:
            
            to_csv('city.csv',city.dict())
            to_csv('weather.csv', weather.dict())
            to_csv('astronomy.csv', astronomy.dict())
            to_csv('metadata.csv', hourly)




In [70]:

state = pd.read_csv("list_of_capitals.csv").dropna()
capitals = state['Capital'].values

it = iter(gen_state(capitals))

        #weather, astronomy, hourly  = parser.parse_weather()

In [75]:
csv_buffer = StringIO()
data = next(it)
parser = ParseFile(json_handler=data)

city = parser.parse_area()
writer = csv.DictWriter(csv_buffer, city.dict())

writer.writeheader()

In [80]:



writer.writerow(city.dict())
file = csv_buffer.getvalue()

In [85]:
csv_buffer.

AttributeError: '_io.StringIO' object has no attribute 'name'

In [68]:
to_s3(city.dict(), 'testcity.csv')

True

In [45]:
import shutil
shutil.rmtree('./testdir', ignore_errors=False, onerror=None)

In [ ]:
os.f